# 120. Backpropagation

## TORCH.AUTOGRAD를 이용한 자동 미분

- autograd 패키지는 Tensor의 모든 연산에 대해 자동 미분을 제공   

- 신경망을 훈련할 때 가장 자주 사용되는 알고리즘은 역전파이다. 이 알고리즘에서 매개 변수 (모델 가중치)는 주어진 매개 변수에 대한 손실 함수의 기울기에 따라 조정된다.  

- 이러한 그래디언트를 계산하기 위해 PyTorch에는 torch.autograd라는 내장 미분 엔진이 있다.

- ``.requires_grad`` 속성을 True 로 설정하면, 그 tensor에서 이뤄진 모든 연산들을 추적(track)하기 시작  

- 계산이 완료된 후 ``.backward()`` 를 호출하여 모든 변화도(gradient)를 자동으로 계산  

- 이 Tensor의 변화도는 ``.grad`` 속성에 누적  

- Tensor가 기록을 추적하는 것을 중단하게 하려면, ``.detach()`` 를 호출하여 연산 기록으로부터 분리(detach)하여 이후 연산들이 추적되는 것을 방지한다.

- 도함수를 계산하기 위해서는 Tensor 의 ``.backward()`` 를 호출

x입력 , 매개변수 , w그리고 손실 함수를 갖는 가장 간단한 단층 신경망을 생각해 보겠습니다 b. PyTorch에서 다음과 같이 정의할 수 있습니다.

In [ ]:
# 가중치 텐서 (5x3), 학습 대상이므로 requires_grad=True
# 편향 텐서 (3차원), 학습 대상이므로 requires_grad=True
# 선형 연산 z = x @ w + b (행렬곱 후 편향 더함)
# Binary Cross Entropy 손실 계산 (로짓 입력 사용)

이 코드는 다음과 같은 계산 그래프를 정의합니다 .

<img src="https://i.imgur.com/Ql8Ff3Z.png" width=450 />

이 네트워크에서 w와 는 최적화해야 할 매개변수b 입니다 . 따라서 해당 변수에 대한 손실 함수의 기울기를 계산할 수 있어야 합니다. 이를 위해 해당 텐서의 속성을 `requires_grad=True`로 설정합니다.  
텐서에 적용하여 계산 그래프를 생성하는 함수는 실제로 클래스 객체입니다 Function. 이 객체는 함수를 순방향 으로 계산하는 방법과 역 전파 단계 에서 미분을 계산하는 방법을 알고 있습니다 . 역전파 함수에 대한 참조는 텐서의 속성에 저장됩니다 .

## 그라디언트 계산 (Gradient Calculation)

신경망에서 매개변수의 가중치를 최적화하려면 **손실 함수(Loss Function)** 를 매개변수에 대해 미분해야 합니다. 즉, 다음 미분값을 계산해야 합니다:

- ∂Loss / ∂w (가중치에 대한 손실 함수의 변화율)
- ∂Loss / ∂b (편향에 대한 손실 함수의 변화율)

이러한 미분을 통해 경사하강법(Gradient Descent) 등의 최적화 알고리즘으로 파라미터를 업데이트할 수 있습니다.

### PyTorch에서의 순서

1. **손실 함수 계산**
```python
loss = loss_fn(output, target)
```

2. **역전파 실행 (Backward Propagation)**
```python
loss.backward()
```

3. **그라디언트 값 확인**
```python
print(w.grad)  # 가중치 w에 대한 손실 함수의 미분값
print(b.grad)  # 편향 b에 대한 손실 함수의 미분값
```

여기서 `x`와 `y`는 고정된 입력(input)과 정답(target) 값이며, `w`와 `b`는 학습해야 할 파라미터입니다.

이 과정을 통해 파라미터가 손실을 줄이는 방향으로 업데이트됩니다.

### 그라디언트 추적 비활성화

기본적으로 requires_grad=True 인 모든 텐서는 계산 기록을 추적하고 기울기 계산을 지원한다. 이를 수행 할 필요가 없는 경우, 즉 네트워크를 통해 순방향 계산만 수행하려는 경우 계산 코드를 torch.inference_mode() 블록으로 둘러 싸서 계산 추적을 중지할 수 있다.

그래디언트 추적을 비활성화 해야하는 경우는 다음과 같다.  

    - pre-train 된 network 를 fine tuning 하는 경우  
    - 기울기를 추적하지 않는 텐서에 대한 계산이 더 효율적이기 때문에 순방향 패스 만 수행할 때 계산 속도를 높인다.
    - 주로 모델의 평가(evaluation)나 추론(inference) 단계에서 사용되며, 계산 속도를 높이고 메모리 사용을 줄이는 데 유용합니다.

- detach()는 특정 텐서에만 적용되어, 해당 텐서가 계산 그래프에서 분리됩니다.  주로 텐서의 일부 값만을 고정시키고 나머지 값들에 대해 그래디언트를 계산하고 싶을 때 사용됩니다.

## 2 개의 feature 를 가진 1 layer + sigmoid activation
<img src="https://i.imgur.com/dQHOqCj.png" width=500 />

In [ ]:
# 입력 텐서 (x1=1.0, x2=2.0)
# 가중치(w1, w2) 및 편향 초기화 (학습 대상)
# 옵티마이저 (경사하강법, 학습률 0.1)
# ----- 순전파 -----
# ----- 역전파 -----
# 기울기 확인 (업데이트 전)
# ----- 경사하강법 적용 (1회) -----
# 업데이트된 파라미터 확인